In [11]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.get('https://bina.az/')
driver.maximize_window()
time.sleep(3)


driver.find_element(By.XPATH, '//*[@id="new_q"]/div[1]/div[2]/div[1]').click()
time.sleep(3)


driver.find_element(By.XPATH, '//*[@id="new_q"]/div[1]/div[2]/div[1]/div[3]/div/ul/li[3]/span').click()
time.sleep(3)

for i in range(5):
    
        driver.execute_script("window.scrollTo(0, 2500)")
        time.sleep(1)
        prices = driver.find_elements(By.CLASS_NAME, 'price-val')
        locations = driver.find_elements(By.CLASS_NAME, 'location')
        names = driver.find_elements(By.CLASS_NAME, 'name')
        city_whens = driver.find_elements(By.CLASS_NAME, 'city_when')

       
        scraped_data = [
    {
        "Price": price.text,
        "Location": location.text,
        "Name": name.text,
        "City/When": city_when.text
    }
    for price, location, name, city_when in zip(prices, locations, names, city_whens)
]

        
        next_button = driver.find_element(By.CLASS_NAME, 'next')
        print(next_button)
        time.sleep(1)
        next_button.click()
        time.sleep(3)  

driver.quit()



<selenium.webdriver.remote.webelement.WebElement (session="c4c64abbe9c7ee32d6948c1d4d916604", element="4695D9A808318D0A6FA9302AB3806DC1_element_207")>
<selenium.webdriver.remote.webelement.WebElement (session="c4c64abbe9c7ee32d6948c1d4d916604", element="4695D9A808318D0A6FA9302AB3806DC1_element_331")>
<selenium.webdriver.remote.webelement.WebElement (session="c4c64abbe9c7ee32d6948c1d4d916604", element="4695D9A808318D0A6FA9302AB3806DC1_element_453")>
<selenium.webdriver.remote.webelement.WebElement (session="c4c64abbe9c7ee32d6948c1d4d916604", element="4695D9A808318D0A6FA9302AB3806DC1_element_574")>
<selenium.webdriver.remote.webelement.WebElement (session="c4c64abbe9c7ee32d6948c1d4d916604", element="4695D9A808318D0A6FA9302AB3806DC1_element_697")>


In [53]:
df=pd.DataFrame(scraped_data)

In [54]:
df.head()

,Price,Location,Name,City/When
0,275 000,İçəri Şəhər m.,,"Bakı, dünən 02:30"
1,156 000,Nəsimi r.,,"Bakı, 19 sentyabr 2023"
2,171 000,Nəsimi r.,,"Bakı, 06 sentyabr 2023"
3,189 000,Azadlıq Prospekti m.,4 otaqlı\n150 m²\n3/3 mərtəbə,"Bakı, 18 sentyabr 2023"
4,176 500,Nəsimi m.,2 otaqlı\n70 m²\n1/9 mərtəbə,"Bakı, dünən 21:08"


In [ ]:
extracted_data = df['Name'].str.extract(r'(\d+) otaqlı\n(\d+) m²\n(\d+/\d+) mərtəbə')


extracted_data.columns = ['Rooms', 'Area', 'Floor']

extracted_data[['CurrentFloor', 'TotalFloors']] = extracted_data['Floor'].str.split('/', expand=True)


df = df.drop(columns=['Name'])


df = pd.concat([df, extracted_data], axis=1)

In [57]:
df.head()

,Price,Location,City/When,Rooms,Area,Floor,CurrentFloor,TotalFloors
0,275 000,İçəri Şəhər m.,"Bakı, dünən 02:30",NaN,NaN,NaN,NaN,NaN
1,156 000,Nəsimi r.,"Bakı, 19 sentyabr 2023",NaN,NaN,NaN,NaN,NaN
2,171 000,Nəsimi r.,"Bakı, 06 sentyabr 2023",NaN,NaN,NaN,NaN,NaN
3,189 000,Azadlıq Prospekti m.,"Bakı, 18 sentyabr 2023",4,150,3/3,3,3
4,176 500,Nəsimi m.,"Bakı, dünən 21:08",2,70,1/9,1,9


In [60]:
df[['City', 'When']] = df['City/When'].str.split(', ', expand=True)

In [66]:
df.head()

,Price,Location,City/When,Rooms,Area,Floor,CurrentFloor,TotalFloors,City,When
0,275 000,İçəri Şəhər m.,"Bakı, dünən 02:30",NaN,NaN,NaN,NaN,NaN,Bakı,NaN
1,156 000,Nəsimi r.,"Bakı, 19 sentyabr 2023",NaN,NaN,NaN,NaN,NaN,Bakı,NaN
2,171 000,Nəsimi r.,"Bakı, 06 sentyabr 2023",NaN,NaN,NaN,NaN,NaN,Bakı,NaN
3,189 000,Azadlıq Prospekti m.,"Bakı, 18 sentyabr 2023",4,150,3/3,3,3,Bakı,NaN
4,176 500,Nəsimi m.,"Bakı, dünən 21:08",2,70,1/9,1,9,Bakı,NaN


In [ ]:
import pandas as pd
from datetime import datetime, timedelta



df[['City', 'When']] = df['City/When'].str.split(', ', expand=True)


month_mapping = {
    "yanvar": "01",
    "fevral": "02",
    "mart": "03",
    "aprel": "04",
    "may": "05",
    "iyun": "06",
    "iyul": "07",
    "avqust": "08",
    "sentyabr": "09",
    "oktyabr": "10",
    "noyabr": "11",
    "dekabr": "12"
}


def convert_to_dd_mm_yyyy(date_string):
    parts = date_string.split()
    
    if len(parts) == 3:
        day = parts[0]
        month_name = parts[1].lower()
        year = parts[2]
        month = month_mapping.get(month_name, "")
        
        if day and month and year:
            return f"{day}/{month}/{year}"
    elif date_string.startswith('dünən'):
        
        yesterday = datetime.now() - timedelta(days=1)
        return yesterday.strftime("%d/%m/%Y")
    
    return None


df['When'] = df['When'].apply(convert_to_dd_mm_yyyy)



In [90]:
df = df.drop(columns=["City/When"])

In [92]:
df=df.drop(columns=['City'])

In [94]:
df=df.drop(columns=['Floor'])

In [ ]:
numeric_columns = ["Rooms", "Area", "CurrentFloor", "TotalFloors"]

for column in numeric_columns:
    df[column] = df[column].astype(float)
    df[column].fillna(df[column].mean(), inplace=True)


int_columns = ["Rooms", "Area", "CurrentFloor", "TotalFloors", 'Price']

for column in int_columns:
    df[column] = df[column].astype(int)

In [104]:
df.head()

,Price,Location,Rooms,Area,CurrentFloor,TotalFloors,When
0,275000,İçəri Şəhər m.,2,74,4,6,20/09/2023
1,156000,Nəsimi r.,2,74,4,6,19/09/2023
2,171000,Nəsimi r.,2,74,4,6,06/09/2023
3,189000,Azadlıq Prospekti m.,4,150,3,3,18/09/2023
4,176500,Nəsimi m.,2,70,1,9,20/09/2023


In [105]:
df['Building_type'] = 'Köhnə tikili'

In [ ]:
column_order = ['When', 'Location', 'Building_type', 'Price', 'Rooms', 'Area', 'TotalFloors', 'CurrentFloor']


df = df.loc[:, column_order]

In [110]:
df.head()

,When,Location,Building_type,Price,Rooms,Area,TotalFloors,CurrentFloor
0,20/09/2023,İçəri Şəhər m.,Köhnə tikili,275000,2,74,6,4
1,19/09/2023,Nəsimi r.,Köhnə tikili,156000,2,74,6,4
2,06/09/2023,Nəsimi r.,Köhnə tikili,171000,2,74,6,4
3,18/09/2023,Azadlıq Prospekti m.,Köhnə tikili,189000,4,150,3,3
4,20/09/2023,Nəsimi m.,Köhnə tikili,176500,2,70,9,1
